In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import cx_Oracle
import time
from bs4 import BeautifulSoup
from urllib.parse import unquote
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 데이터베이스 연결
connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()
cursor.execute('SELECT id, search_name FROM zero')

# id와 search_name을 모두 포함하는 리스트 생성
data_list = [(row[0], row[1]) for row in cursor.fetchall()]
cursor.close()
connection.close()

# Selenium 웹 드라이버 설정
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 네이버 지도 URL
url = "https://map.naver.com/v5/search"
driver.get(url)

time.sleep(3)

def search_on_naver_map(keyword):
    search_box = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search_box.clear()
    search_box.send_keys(keyword)  # 검색어 입력
    search_box.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    time.sleep(10)

def click_photo_tab():
    try:
        tabs = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a/span')
        photo_tab_found = False
        for tab in tabs:
            if "사진" in tab.text:
                tab.click()
                photo_tab_found = True
                time.sleep(3)  # 클릭 후 페이지 로딩 대기
                break
        
        if not photo_tab_found:
            print("사진 탭을 찾을 수 없습니다.")
            return False
        return True
    except Exception as e:
        print(f"사진 탭을 클릭하는 중 오류 발생: {e}")
        return False


def get_photo_contents():
    photo_contents = []
    if not click_photo_tab():
        return photo_contents  # 사진 탭을 클릭하지 못했으면 빈 리스트 반환
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        images = soup.find_all("img", class_="K0PDV")  # 모든 사진 요소를 찾습니다.
        for img in images:
            image_src = img.get('src')  # 이미지의 URL을 가져옴
            if image_src:
                # URL 디코딩 및 필요한 경우 처리
                decoded_url = image_src.split('&src=')[1] if '&src=' in image_src else image_src
                photo_contents.append(decoded_url)
    except Exception as e:
        print(f"Failed to extract photos: {e}")
    
    return photo_contents



# 데이터베이스 연결 재설정
connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()

# 리스트의 각 항목에 대해 검색 수행 및 데이터베이스 업데이트
for id, name in data_list:
    search_on_naver_map(name)
    
    # 사진 검색 및 저장
    photo_urls = get_photo_contents()
    image_data_list = []
    for url in photo_urls:
        response = requests.get(url)
        if response.status_code == 200:
            image_data_list.append(response.content)
        else:
            image_data_list.append(None)  # 다운로드 실패시 None 추가

    cursor.execute(f"""
        INSERT INTO zero_images (id, img_num, img1, img2, img3, img4, img5, img6, img7, img8)
        VALUES (:id, Zero_img_seq.NEXTVAL, :img1, :img2, :img3, :img4, :img5, :img6, :img7, :img8)
    """, {
        'id': id,
        'img1': image_data_list[0] if len(image_data_list) > 0 else None,
        'img2': image_data_list[1] if len(image_data_list) > 1 else None,
        'img3': image_data_list[2] if len(image_data_list) > 2 else None,
        'img4': image_data_list[3] if len(image_data_list) > 3 else None,
        'img5': image_data_list[4] if len(image_data_list) > 4 else None,
        'img6': image_data_list[5] if len(image_data_list) > 5 else None,
        'img7': image_data_list[6] if len(image_data_list) > 6 else None,
        'img8': image_data_list[7] if len(image_data_list) > 7 else None
    })

    connection.commit()

# 연결 종료
cursor.close()
connection.close()
